In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import imageio
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
scale_factor = 4
resize_method = tf.image.ResizeMethod.BICUBIC

builder_kwargs = {
    'resize_method': resize_method,
    'scale': scale_factor,
    'antialias': True
}
images = tfds.load(name="set14", split="test", builder_kwargs=builder_kwargs) \
                .map(datasets.map_on_dict(datasets.to_float32)) \
                .map(datasets.map_on_dict(datasets.from_255_to_1_range))

In [ ]:
image_dict = datasets.get_one_example(images, index=2)
image_hr = image_dict['hr'][None, ...]
image_lr = image_dict['lr'][None, ...]
visualize.draw_images(image_hr)

In [ ]:
h, w = tf.shape(image_hr)[1:3]
image_bicubic = tf.image.resize(image_lr, (h, w), method='bicubic')
bicubic_psnr = dppp.psnr(image_hr, image_bicubic).numpy()[0]
bicubic_fsim = dppp.fsim(image_hr, image_bicubic).numpy()[0]
print(f"Bicubic PSNR: {bicubic_psnr:0.2f}, FSIM: {bicubic_fsim:0.4f}")
visualize.draw_images(image_bicubic)

# Run different Methods and Models

In [ ]:
model_methods = [
    ('dae_0.04314', 'dmsp', None),
    ('drcnn_0.05', 'dmsp', None),
    ('drunet+_0.0-0.2', 'dmsp', 0.1),
    ('drugan+_0.0-0.2', 'dmsp', 0.1),
    ('drunet+_0.0-0.2', 'hqs', None),
    ('drugan+_0.0-0.2', 'hqs', None),
]

In [ ]:
all_reconstructed = []
all_psnr = []
all_fsim = []

for model_name, method, denoiser_stddev in model_methods:
    model_path = os.path.join('models', f'{model_name}.h5')
    denoiser, (_, max_denoiser_stddev) = dppp.load_denoiser(model_path)
    if denoiser_stddev is None:
        denoiser_stddev = max_denoiser_stddev
        
    print(f"Running {method} for {model_name}...")
    if method == 'dmsp':
        rec = dppp.dmsp_super_resolve(image_lr, scale_factor,
                                      denoiser, denoiser_stddev)
        
    if method == 'hqs':
        rec = dppp.hqs_super_resolve(image_lr, scale_factor,
                                     denoiser, max_denoiser_stddev)
    
    psnr = dppp.psnr(image_hr, rec).numpy()[0]
    fsim = dppp.fsim(image_hr, rec).numpy()[0]
    print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")
    
    all_reconstructed.append(rec)
    all_psnr.append(psnr)
    all_fsim.append(fsim)

# Visualize

In [ ]:
visualize.draw_images([image_hr[0], image_bicubic[0], *[x[0] for x in all_reconstructed]], ncol=4, figsize=(50, 40))

# Export

In [ ]:
export_dir = os.path.join('visualize', 'super_res')

In [ ]:
os.makedirs(export_dir)

def export_path(x):
    return os.path.join(export_dir, x)

def to_uint8(x):
    return tf.cast(tf.clip_by_value(x, 0, 1) * 255, tf.uint8)

def write_to(file_name, img, psnr, fsim):
    imageio.imwrite(export_path(f'{file_name}.png'), to_uint8(img[0]))
    with open(export_path(f'{file_name}.txt'), 'w') as f:
        f.write(f"PSNR: {psnr}, FSIM: {fsim}")

#### Images

# High resolution
imageio.imwrite(export_path('gt.png'), to_uint8(image_hr[0]))

# Bicubic
write_to('bicubic', image_bicubic, bicubic_psnr, bicubic_fsim)

# Reconstructed
for mm, rec, psnr, fsim in zip(model_methods, all_reconstructed, all_psnr, all_fsim):
    model_name, method, denoiser_stddev = mm
    write_to(f'{method}-{model_name}', rec, psnr, fsim)

# Highlight a Region of Interest

Also add results from other methods before running these cells

In [ ]:
import os
import imageio
from highlight import highlight, highlight_and_save

### Preview and Region Selection

In [ ]:
image = imageio.imread(os.path.join(export_dir, 'gt.png'))

# Location of the region of interest
center = (120,120)
extent = 30

# Size of the box
rel_width = 0.7

# Positioning of the box
border = 0.01
top = False
right = False
label_border = 0.07
fontsize=12

fig = highlight(image, center, extent, rel_width, top, right, border, label_border, fontsize)

### Highlight

In [ ]:
highlight_and_save(export_dir, center, extent, rel_width, top, right, border, label_border, fontsize)